# Projekt 6: Przewidywanie struktury drugorzędowej RNA

In [1]:
# works for python 3.8.19 :')
# %pip install tensorflow pandas tqdm argparse six

In [2]:
import os

os.chdir("./SPOT-RNA")

cores = max(1, os.cpu_count() - 1)

In [3]:
!python3 SPOT-RNA.py  --inputs sample_inputs/single_seq.fasta  --outputs 'outputs/'  --cpu {cores}

>> Opening FASTA file...
>> Converting FASTA file from multiline to single line and writing to file.
>> Done!

Preparing tfr records file for SPOT-RNA:
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 35.10it/s]

Predicting for SPOT-RNA model 0
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.87it/s]

Predicting for SPOT-RNA model 1
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.82it/s]

Predicting for SPOT-RNA model 2
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.18it/s]

Predicting for SPOT-RNA model 3
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]

Predicting for SPOT-RNA model 4
100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.28s/it]

Post Processing and Saving Output

Finished!

Processsing Time 10.213176012039185 seconds
